In [ ]:
# default_exp bayesian
%load_ext autoreload
%autoreload 2

from nbdev import showdoc 
from utilities.ipynb_docgen import *

# Bayesian Blocks

> Partition a light curve with the Bayesian Block algorithm

The algorithm depends on a 'fitness' function of the light curve, an evaluation of the 
likelihoods for a set of sequential cells. There are two such, using the number of counts, and the Kerr likelihood.

- `CountFitness`
- `LikelihoodFitness`

See the [Bayesian Block reference](https://arxiv.org/pdf/1207.5578.pdf)

In [ ]:
#export
import os
import numpy as np
import pandas as pd
from astropy.stats.bayesian_blocks import FitnessFunc
from light_curves.lightcurve import get_lightcurve, fit_cells, flux_plot
from light_curves.cells import get_cells, partition_cells


In [ ]:
#collapse_hide
from light_curves.config import Config, PointSource

lcs ={}
def data_setup(lcs = lcs, mjd_query='54750<t<54855', names=['Geminga','3C 279']):
    """
    ## Generate data sets for an AGN and a pulsar
    {printout}
    
    Choose the time interval, {mjd_query} ({days} days) to bracket a modest flare of the AGN.
     
    <table>
    <tr> <td>Pulsar</td><td>AGN</td></tr>
    <tr>
    <td>{fig1}</td> <td>{fig2}</td>
    </tr>
    </table>
    """
    
    from light_curves.config import Config,  PointSource
    from light_curves.lightcurve import get_lightcurve, flux_plot
    config = Config()
    figs=[]
    plt.rc('font', size=20)
    with capture_print('printout') as printout:
        for i,name in enumerate(names):
            lcfull = get_lightcurve(config,  PointSource(name))
            lc = lcs[name] = lcfull.query(mjd_query) if mjd_query else lcfull
            fig= flux_plot(config,lc, fignum=i, title=name)
            figs.append(figure(fig, width=400))
    fig1, fig2 = figs
    mjd_query = mjd_query.replace('<', '&lt;')
    days = len(lc)
    return locals()

if Config().valid:
    nbdoc(data_setup, lcs, mjd_query='54750<t<54855', )

## Generate data sets for an AGN and a pulsar
<details class="descripton" ><summary data-open="Hide " data-close="Show "> printout </summary> <p style="margin-left: 5%"><pre>using cache with key "lightcurve_Geminga", exists: True<br>using cache with key "lightcurve_3C 279", exists: True<br></pre></p> </details>

Choose the time interval, 54750&lt;t&lt;54855 (105 days) to bracket a modest flare of the AGN.
 
<table>
<tr> <td>Pulsar</td><td>AGN</td></tr>
<tr>
<td><div class="nbdoc_image">
<a href="images/data_setup_fig_02.png"><figure>
   <img src="images/data_setup_fig_02.png" alt="Figure 2 at images/data_setup_fig_02.png" width=400> 
</figure></a></div>
</td> <td><div class="nbdoc_image">
<a href="images/data_setup_fig_03.png"><figure>
   <img src="images/data_setup_fig_03.png" alt="Figure 3 at images/data_setup_fig_03.png" width=400> 
</figure></a></div>
</td>
</tr>
</table>


In [ ]:
#export
class CountFitness(FitnessFunc):
    """
    Adapted version of a astropy.stats.bayesian_blocks.FitnessFunc
    Considerably modified to give the `fitness function` access to the cell data.
    
    Implements the Event model using exposure instead of time.

    """

    def __init__(self, lc, p0=0.05,):
        """
        - lc  -- a LightCurve data table, with  exposure (e) and counts (n),
            as well as a representation of the likelihood for each cell
        - p0 --
        """
        self.p0=p0
        self.df= df= lc
        N = self.N = len(df)
        # Invoke empirical function from Scargle 2012
        self.ncp_prior = self.p0_prior(N)

        #actual times for bin edges
        t = df.t.values
        dt = df.tw.values/2
        self.mjd = np.concatenate([t-dt, [t[-1]+dt[-1]] ] ) # put one at the end
        self.name = self.__class__.__name__
        self.setup()

    def setup(self):
        df = self.df

        # counts per cell
        self.nn = df.n.values
        assert min(self.nn)>0, 'Attempt to Include a cell with no contents'

        # edges and block_length use exposure as "time"
        e = df.e.values
        self.edges = np.concatenate([[0], np.cumsum(e)])
        self.block_length = self.edges[-1] - self.edges

    def __str__(self):
        
        return f'{self.name}: {self.N} cells, spanning {self.block_length[0]:.1f} days, prior={self.ncp_prior:.1f}'
        
    def __call__(self, R):
        """ The fitness function needed for BB algorithm
        For cells 0..R return array of length R+1 of the maximum log likelihoods for combined cells
        0..R, 1..R, ... R
        """
        # exposures and corresponding counts
        w_k = self.block_length[:R + 1] - self.block_length[R + 1]
        N_k = np.cumsum(self.nn[:R + 1][::-1])[::-1]

        # Solving eq. 26 from Scargle 2012 for maximum $\lambda$ gives
        return N_k * (np.log(N_k) - np.log(w_k))

    def fit(self):
        """Fit the Bayesian Blocks model given the specified fitness function.
        Refactored version using code from bayesian_blocks.FitnesFunc.fit
        Returns
        -------
        edges : ndarray
            array containing the (M+1) edges, in MJD units, defining the M optimal bins
        """
        # This is the basic Scargle algoritm, copied almost verbatum
        # ---------------------------------------------------------------

        # arrays to store the best configuration
        N = self.N
        best = np.zeros(N, dtype=float)
        last = np.zeros(N, dtype=int)

        # ----------------------------------------------------------------
        # Start with first data cell; add one cell at each iteration
        # ----------------------------------------------------------------
        for R in range(N):

            # evaluate fitness function
            fit_vec = self(R)

            A_R = fit_vec - self.ncp_prior
            A_R[1:] += best[:R]

            i_max = np.argmax(A_R)
            last[R] = i_max
            best[R] = A_R[i_max]

        # ----------------------------------------------------------------
        # Now find changepoints by iteratively peeling off the last block
        # ----------------------------------------------------------------
        change_points = np.zeros(N, dtype=int)
        i_cp = N
        ind = N
        while True:
            i_cp -= 1
            change_points[i_cp] = ind
            if ind == 0:
                break
            ind = last[ind - 1]
        change_points = change_points[i_cp:]

        return self.mjd[change_points]

In [ ]:
#hide
showdoc.show_doc(CountFitness)

<h2 id="CountFitness" class="doc_header"><code>class</code> <code>CountFitness</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CountFitness</code>(**`lc`**, **`p0`**=*`0.05`*) :: `FitnessFunc`

Adapted version of a astropy.stats.bayesian_blocks.FitnessFunc
Considerably modified to give the `fitness function` access to the cell data.

Implements the Event model using exposure instead of time.

In [ ]:
#collapse_hide
def doc_countfitness( fitness, light_curve_dict, source_name):
    """
    ### {class_name} test with source {source_name}
         
    Create object: `bbfitter = {class_name}(lc)`
    
    Object description:   {bbfitter}
    
    Then `bbfitter({n})` returns the values
        {values}
   
    Finally, the partition algorithm, 'bbfitter.fit()' returns {cffit}
    
    """
    
    lc = light_curve_dict[source_name]
    bbfitter = fitness(lc)
    class_name = bbfitter.name
    n = 10
    values  = np.array(bbfitter(n)).round()    
    cffit = bbfitter.fit()
    
    return locals()

In [ ]:
#hide
from light_curves.config import Config,  PointSource
from light_curves.lightcurve import get_lightcurve, flux_plot
if Config().valid:
    nbdoc(doc_countfitness, CountFitness, light_curve_dict = lcs, source_name='Geminga')
    nbdoc(doc_countfitness, CountFitness, light_curve_dict = lcs, source_name='3C 279')
     

### CountFitness test with source Geminga
     
Create object: `bbfitter = CountFitness(lc)`

Object description:   CountFitness: 105 cells, spanning 131.0 days, prior=4.9

Then `bbfitter(10)` returns the values
    [14517. 13066. 11631. 10160.  9106.  7634.  6573.  5174.  3960.  2747.
  1451.]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54788. 54855.]


### CountFitness test with source 3C 279
     
Create object: `bbfitter = CountFitness(lc)`

Object description:   CountFitness: 105 cells, spanning 138.0 days, prior=4.9

Then `bbfitter(10)` returns the values
    [1657. 1431. 1268. 1145.  919.  811.  648.  510.  379.  266.  148.]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54754. 54785. 54790. 54807. 54827. 54855.]


In [ ]:
#export
class LikelihoodFitness(CountFitness):
    """ Fitness function that uses the full likelihood
    """
    
    def __init__(self, lc,  p0=0.05, npt=25):
        self.npt = npt
        super().__init__(lc, p0)
        
    def setup(self):
        df = self.df
        N = self.N
        
        def liketable(prep):
            return prep.create_table(self.npt)
        
        self.tables = df.fit.apply(liketable).values

    def __str__(self):
        return f'{self.__class__.__name__}: {self.N} cells,  prior={self.ncp_prior:.1f}'

    def __call__(self, R):
        
        a, y  = self.tables[R]
        x = np.linspace(*a)
        y = np.zeros(self.npt)
        rv = np.empty(R+1)
        for i in range(R, -1, -1): 
            a, yi = self.tables[i]
            xi = np.linspace(*a)
            y += np.interp(x, xi, yi, left=-np.inf, right=-np.inf)
            amax = np.argmax(y)
            rv[i] =y[amax]
        return rv    

In [ ]:
#hide
showdoc.show_doc(LikelihoodFitness)

<h2 id="LikelihoodFitness" class="doc_header"><code>class</code> <code>LikelihoodFitness</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>LikelihoodFitness</code>(**`lc`**, **`p0`**=*`0.05`*, **`npt`**=*`25`*) :: [`CountFitness`](/light_curves/bayesian.html#CountFitness)

Fitness function that uses the full likelihood
    

In [ ]:
if Config().valid:
    nbdoc(doc_countfitness, LikelihoodFitness, light_curve_dict = lcs, source_name='Geminga')
    nbdoc(doc_countfitness, LikelihoodFitness, light_curve_dict = lcs, source_name='3C 279')
     

### LikelihoodFitness test with source Geminga
     
Create object: `bbfitter = LikelihoodFitness(lc)`

Object description:   LikelihoodFitness: 105 cells,  prior=4.9

Then `bbfitter(10)` returns the values
    [-4. -4. -2. -2. -1. -1. -0. -0. -0. -0. -0.]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54793. 54855.]


### LikelihoodFitness test with source 3C 279
     
Create object: `bbfitter = LikelihoodFitness(lc)`

Object description:   LikelihoodFitness: 105 cells,  prior=4.9

Then `bbfitter(10)` returns the values
    [-9. -9. -8. -7. -6. -6. -5. -4. -3. -3. -0.]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54779. 54785. 54789. 54809. 54827. 54843. 54855.]


In [ ]:
#export
def get_bb_partition(config, lc, fitness_class=CountFitness, p0=0.05):    

    """Perform Bayesian Block partition of the cells found in a light curve
    
    - lc : input light curve
    - fitness_func : 
    
    return edges for partition
    """
    cells = lc
    assert 'fit' in cells.columns, 'Expect the dataframe ho have the Poisson representation'
    assert issubclass(fitness_class,CountFitness), 'fitness_class wrong'
    # Now run the astropy Bayesian Blocks code using my version of the 'event' model
    fitness = fitness_class(lc, p0=p0)
    edges = fitness.fit() 

    if config.verbose>0:
        print(f'Partitioned {fitness.N} cells into {len(edges)-1} blocks, with prior {fitness.ncp_prior:.1f}'\
              f' using {fitness.__class__.__name__} ' )
    return edges

In [ ]:
#hide
showdoc.show_doc(get_bb_partition)

<h4 id="get_bb_partition" class="doc_header"><code>get_bb_partition</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_bb_partition</code>(**`config`**, **`lc`**, **`fitness_class`**=*`CountFitness`*, **`p0`**=*`0.05`*)

Perform Bayesian Block partition of the cells found in a light curve

- lc : input light curve
- fitness_func : 

return edges for partition

In [ ]:
#collapse_hide

def test_bb(lcs, name, fitness):
    """
    ### Display results of the BB partition for {name}
    
    {lc_fig}
    """

    config = Config()
     
    lc = lcs[name]
    edges = get_bb_partition(config, lc, fitness) 
    lc_fig = flux_plot(config, lc, title=f'{name} partition with {fitness.__name__}')
    lc_fig.width=400
    ax = lc_fig.axes[0]
    edges = np.concatenate([edges, [edges[-1]] ])
    for  i,t in enumerate(edges[::2]):
        if 2*i+1==len(edges): break
        t2 = edges[2*i+1]
        ax.axvspan(t, t2, color='lightcyan')
    for t in edges:
        ax.axvline(t, ls=':', color='cyan')
    return locals()

if Config().valid:
    nbdoc(test_bb, lcs, '3C 279', CountFitness)
    nbdoc(test_bb, lcs, '3C 279', LikelihoodFitness)

Partitioned 105 cells into 6 blocks, with prior 4.9 using CountFitness 


### Display results of the BB partition for 3C 279

<div class="nbdoc_image">
<a href="images/test_bb_fig_01.png"><figure>
   <img src="images/test_bb_fig_01.png" alt="Figure 1 at images/test_bb_fig_01.png" width=400> 
</figure></a></div>



Partitioned 105 cells into 7 blocks, with prior 4.9 using LikelihoodFitness 


### Display results of the BB partition for 3C 279

<div class="nbdoc_image">
<a href="images/test_bb_fig_01.png"><figure>
   <img src="images/test_bb_fig_01.png" alt="Figure 1 at images/test_bb_fig_01.png" width=400> 
</figure></a></div>



## Test fitting the new cells from the partition

In [ ]:
source = PointSource('Geminga')
config = Config()
cells = get_cells(config, source)

restored photons_Geminga from cache

	Selected 1313726 photons within 5 deg of  (195.13,4.27)
	Energies: 100.0-1000000 MeV
	Dates:    2008-08-04 15:46 - 2019-08-03 01:17
	MJD  :    54682.7          - 58698.1         
Load weights from file /mnt/c/users/thbur/OneDrive/fermi/weight_files/Geminga_weights.pkl
	Found: PSR J0633+1746 at (195.14, 4.27)
	Applyng weights: 240 / 1313726 photon pixels are outside weight region
	233109 weights set to NaN
using cache with key "exposure_Geminga", exists: True
Time bins: 4015 intervals of 1 days, in range (54683.0, 58698.0)


Partition it:

In [ ]:
lcx =  lcs['Geminga']
edges = get_bb_partition(config, lcx, LikelihoodFitness) 
#lcx = get_lightcurve(config,  source, edges)

Partitioned 105 cells into 2 blocks, with prior 4.9 using LikelihoodFitness 


Use the edges to partition the original cells 

In [ ]:
pcells = partition_cells(config, cells, edges);

Likelihood fits to the new cells 

In [ ]:
fit_cells(config, pcells, )

Loaded 2 / 2 cells with exposure > 0.3 for fitting


,t,tw,n,e,fit
0,54771.5,43.0,14080,1.263585,"flux: 0.953[1+0.012-0.012], limit: 0.97, ts: ..."
1,54824.0,62.0,20872,1.236235,"flux: 1.019[1+0.009-0.009], limit: 1.04, ts: ..."


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted 12_roadmap.ipynb.
Converted 13_kerr_comparison.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Wed Dec 16 12:04:39 PST 2020
